In [ ]:
import pandas as pd
import nltk
import sklearn
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import sys
sys.path.insert(0, '/content/drive/My Drive/Bachelor Scriptie KI/Programming/Notebooks')

In [ ]:
import multi_class_performance_eval as mce

In [ ]:
#Setting the random seeds for reproducability
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Bachelor Scriptie KI/FinancialPhraseBank-v1.0/train.csv", sep=",", names=["Sentence", "Sentiment"], encoding="utf-8", skiprows=[0])
test = pd.read_csv("/content/drive/My Drive/Bachelor Scriptie KI/FinancialPhraseBank-v1.0/test.csv", sep=",", names=["Sentence", "Sentiment"], encoding="utf-8", skiprows=[0])
val = pd.read_csv("/content/drive/My Drive/Bachelor Scriptie KI/FinancialPhraseBank-v1.0/validation.csv", sep=",", names=["Sentence", "Sentiment"], encoding="utf-8", skiprows=[0])

In [ ]:
def label_distribution(dataframe):
  """
  Determine the distribution of labels in the dataframe.

  param dataframe: Pandas DataFrame contains the sentences and sentiment labels.
  """
  total = 0
  pos = 0
  neu = 0
  neg = 0

  for label in dataframe.Sentiment:
    if label == 0:
      neg += 1
    elif label == 1:
      neu += 1
    else:
      pos += 1
    total += 1
  return pos, neu, neg, total

In [ ]:
pos, neu, neg, total = label_distribution(train)
print("Positive: ", str(round(pos/total*100, 2)))
print("Neutral: ", str(round(neu/total*100, 2)))
print("Negative: ", str(round(neg/total*100, 2)))

Positive:  28.2
Neutral:  59.6
Negative:  12.2


In [ ]:
pos, neu, neg, total = label_distribution(test)
print("Positive: ", str(round(pos/total*100, 2)))
print("Neutral: ", str(round(neu/total*100, 2)))
print("Negative: ", str(round(neg/total*100, 2)))

Positive:  26.8
Neutral:  58.76
Negative:  14.43


In [ ]:
pos, neu, neg, total = label_distribution(val)
print("Positive: ", str(round(pos/total*100, 2)))
print("Neutral: ", str(round(neu/total*100, 2)))
print("Negative: ", str(round(neg/total*100, 2)))

Positive:  28.87
Neutral:  58.56
Negative:  12.58


In [ ]:
X_train = train.Sentence
X_test = test.Sentence
X_val = val.Sentence

In [ ]:
#The categorical cross-entropy loss expects the sentiments to be one-hot encoded.
y_train = tf.keras.utils.to_categorical(train.Sentiment.tolist())
y_test = tf.keras.utils.to_categorical(test.Sentiment.tolist())
y_val = tf.keras.utils.to_categorical(val.Sentiment.tolist())

In [ ]:
#Load the BERT preprocesser and encoder into KerasLayers.
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def create_BERT_CNN(preprocesser, encoder, kernel_size, filters, activation, padding):
  """
  Creates a Tensorflow model that incorporates BERT preprocessing and encoding layers.
  Adds a CNN behind it and a Dense layer with 3 neurons for classification.

  param preprocesser: (KerasLayer) a layer containing the BERT preprocesser.
  param encoder: (KerasLayer) a layer containing the BERT encoder.
  param kernel_size: (int) the size of the kernel used in the Convolutional layer.
  param filters: (int) the amount of filters used in the Convolutional layer.
  param activation: (str) activation function to be used in the Convolutional layer.
  param padding: (str) padding used in Convolutional layer.
  """
  #Define the BERT layers
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
  preprocessing_bert = preprocesser(text_input)
  encoder_bert = encoder(preprocessing_bert)
  outputs = encoder_bert['sequence_output']

  #Define the CNN that uses the BERT embeddings
  conv = tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, padding=padding, activation=activation, name="conv1d")(outputs)
  conv = tf.keras.layers.GlobalMaxPool1D(name="pool")(conv)

  #Define Dense output layer
  ff = tf.keras.layers.Dense(3, activation='softmax', name="output")(conv)

  classifier = tf.keras.Model(inputs=[text_input], outputs=[ff])
  return classifier

#Fine-tuning of hyperparameters

In [ ]:
kernel_sizes = [1, 2, 4, 8, 10]
filters = [64, 128, 256, 512]

for size in kernel_sizes:
  for filter in filters:
    #Create the model and compile it.
    model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=size, filters=filter, activation="relu", padding="same")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    #We make use of early stopping and save the best weights.
    early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5)
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./BERT_conv_model", save_best_only=True)

    print("Training model with kernel size ", str(size), " and filters ", str(filter), "\n \n \n")

    hist = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop, checkpoint_cb])

    train_acc_hist = hist.history['accuracy']
    train_acc = 0
    count = 0
    for i in train_acc_hist:
      train_acc += i
      count += 1
    train_acc = train_acc / count
    print("Training accuracy was: ", str(train_acc))

    val_acc_hist = hist.history['val_accuracy']
    val_acc = 0
    count = 0
    for i in val_acc_hist:
      val_acc += i
      count += 1
    val_acc = val_acc / count
    print("Validation accuracy was: ", str(val_acc))

    #Roll back to model found performing best on validation set during training.
    model.load_weights("./BERT_conv_model")

    #Generate predictions using this model
    predictions = model.predict(X_test)

    #Convert the one-hot encoded predictions into a number.
    y_pred = []
    for pred in predictions:
      label = np.argmax(pred)
      y_pred.append(label)

    #Convert the one-hot encoded real sentiment labels into numbers.
    y_real = []
    for encoding in y_test:
      label = np.argmax(encoding)
      y_real.append(label)

    #Evaluate performance of the model using the current set of hyperparameters.
    evaluation = mce.evaluate_performance(y_pred, y_real)

    print("Tested model with kernel size ", str(size), " and filters ", str(filter), " on the test set")
    for metric in evaluation:
      print(metric, ": ", evaluation[metric], "\n")
    print("\n \n \n")






In [ ]:
#Further fine-tuning of the hyperparameters
kernel_sizes = [1, 2, 3, 4]
filters = [200, 220, 256, 280, 300]

for size in kernel_sizes:
  for filter in filters:
        model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=size, filters=filter, activation="relu", padding="same")
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5)
        checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./BERT_conv_model", save_best_only=True)

        print("Training model with kernel size ", str(size), " and filters ", str(filter), "\n \n \n")

        hist = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop, checkpoint_cb])

        train_acc_hist = hist.history['accuracy']
        train_acc = 0
        count = 0
        for i in train_acc_hist:
          train_acc += i
          count += 1
        train_acc = train_acc / count
        print("Training accuracy was: ", str(train_acc))

        val_acc_hist = hist.history['val_accuracy']
        val_acc = 0
        count = 0
        for i in val_acc_hist:
          val_acc += i
          count += 1
        val_acc = val_acc / count
        print("Validation accuracy was: ", str(val_acc))

        #Roll back to model found performing best on validation set during training.
        model.load_weights("./BERT_conv_model")

        #Generate predictions using this model
        predictions = model.predict(X_test)

        y_pred = []
        for pred in predictions:
          label = np.argmax(pred)
          y_pred.append(label)

        y_real = []
        for encoding in y_test:
          label = np.argmax(encoding)
          y_real.append(label)

        #Evaluate performance of the model using the current set of hyperparameters
        evaluation = mce.evaluate_performance(y_pred, y_real)

        print("Tested model with kernel size ", str(size), " and filters ", str(filter), " on the test set")
        for metric in evaluation:
          print(metric, ": ", evaluation[metric], "\n")
        print("\n \n \n")

In [ ]:
#Tune learning rate, the standard is 0.001 according to documentation for Adam optimizer.
learning_rates = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3]
for lr in learning_rates:
    model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=1, filters=256, activation="relu", padding="same")
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

    early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5)
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./BERT_conv_model", save_best_only=True)

    print("Training model with learning rate: ", str(lr), "\n \n \n")

    hist = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop, checkpoint_cb])

    train_acc_hist = hist.history['accuracy']
    train_acc = 0
    count = 0
    for i in train_acc_hist:
      train_acc += i
      count += 1
    train_acc = train_acc / count
    print("Training accuracy was: ", str(train_acc))

    val_acc_hist = hist.history['val_accuracy']
    val_acc = 0
    count = 0
    for i in val_acc_hist:
      val_acc += i
      count += 1
    val_acc = val_acc / count
    print("Validation accuracy was: ", str(val_acc))

    #Roll back to model found performing best on validation set during training.
    model.load_weights("./BERT_conv_model")

    #Generate predictions using this model.
    predictions = model.predict(X_test)

    y_pred = []
    for pred in predictions:
      label = np.argmax(pred)
      y_pred.append(label)

    y_real = []
    for encoding in y_test:
      label = np.argmax(encoding)
      y_real.append(label)

    #Evaluate performance of the model using the current set of hyperparameters.
    evaluation = mce.evaluate_performance(y_pred, y_real)

    print("Tested model with learning rate  ", str(lr), " on the test set")
    for metric in evaluation:
      print(metric, ": ", evaluation[metric], "\n")
    print("\n \n \n")

In [ ]:
#Check the effect of dropout.
dropout_chances = [0.05, 0.1, 0.3, 0.5]
for chance in dropout_chances:
    model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=1, filters=220, activation="relu", padding="same", dropout=chance)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode="max", patience=5)

    print("\n \n", "Training model with dropout ", str(chance), "\n \n \n")

    hist = model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop])

    train_acc_hist = hist.history['accuracy']
    train_acc = 0
    count = 0
    for i in train_acc_hist:
      train_acc += i
      count += 1
    train_acc = train_acc / count
    print("Training accuracy was: ", str(train_acc))

    val_acc_hist = hist.history['val_accuracy']
    val_acc = 0
    count = 0
    for i in val_acc_hist:
      val_acc += i
      count += 1
    val_acc = val_acc / count
    print("Validation accuracy was: ", str(val_acc))

In [ ]:
#Check the influence of different activation functions.
activation_functions = ["relu", "elu"]

for activation in activation_functions:
    model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=1, filters=220, activation=activation, padding="same", dropout=chance)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode="max", patience=5)

    print("\n \n", "Training model with activation function ", str(activation), "\n \n \n")

    hist = model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop])

    train_acc_hist = hist.history['accuracy']
    train_acc = 0
    count = 0
    for i in train_acc_hist:
      train_acc += i
      count += 1
    train_acc = train_acc / count
    print("Training accuracy was: ", str(train_acc))

    val_acc_hist = hist.history['val_accuracy']
    val_acc = 0
    count = 0
    for i in val_acc_hist:
      val_acc += i
      count += 1
    val_acc = val_acc / count
    print("Validation accuracy was: ", str(val_acc))

In [ ]:
#Check the effect of different padding settings.
paddings = ["valid", "same", "causal"]
for padding in paddings:
    model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=1, filters=220, activation="relu", padding=padding, dropout=chance)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode="max", patience=5)

    print("\n \n", "Training model with padding ", str(padding), "\n \n \n")

    hist = model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop])

    train_acc_hist = hist.history['accuracy']
    train_acc = 0
    count = 0
    for i in train_acc_hist:
      train_acc += i
      count += 1
    train_acc = train_acc / count
    print("Training accuracy was: ", str(train_acc))

    val_acc_hist = hist.history['val_accuracy']
    val_acc = 0
    count = 0
    for i in val_acc_hist:
      val_acc += i
      count += 1
    val_acc = val_acc / count
    print("Validation accuracy was: ", str(val_acc))

In [ ]:
def create_BERT_CNN_doubleblock(preprocesser, encoder, kernel_size, filters, activation, padding):
  """
  Creates a Tensorflow model with BERT embeddings, but this time with 2 blocks of Conv1D + GlobalMaxPool1D layers.
  param preprocesser: (KerasLayer) a layer containing the BERT preprocesser.
  param encoder: (KerasLayer) a layer containing the BERT encoder.
  param kernel_size: (int) the size of the kernel used in the Convolutional layer.
  param filters: (int) the amount of filters used in the Convolutional layer.
  param activation: (str) activation function to be used in the Convolutional layer.
  param padding: (str) padding used in Convolutional layer.
  """
  #Define the BERT layers
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
  preprocessing_bert = preprocesser(text_input)
  encoder_bert = encoder(preprocessing_bert)
  outputs = encoder_bert['sequence_output']

  #Define the CNN that uses the BERT embeddings
  conv = tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, padding=padding, activation=activation, name="conv1d.1")(outputs)
  conv = tf.keras.layers.MaxPool1D(name="pool1")(conv)
  conv = tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=(filters/2), padding=padding, activation=activation, name="conv1d.2")(conv)
  conv= tf.keras.layers.GlobalMaxPool1D(name="pool2")(conv)

  #Define Dense output layer
  ff = tf.keras.layers.Dense(3, activation='softmax', name="output")(conv)

  classifier = tf.keras.Model(inputs=[text_input], outputs=[ff])
  return classifier

In [ ]:
double_conv_classifier = create_BERT_CNN_doubleblock(bert_preprocess, bert_encoder, 1, 256, "relu", "same")

In [ ]:
double_conv_classifier.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text_input[0][0]']             
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0003)

double_conv_classifier.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./BERT_conv_model_doubleblock", save_best_only=True)

In [ ]:
print("Training model with 2 convolutional layers \n \n")
hist = double_conv_classifier.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop, checkpoint_cb])

Training model with 2 convolutional layers 
 

Epoch 1/50
122/122 [==============================] - ETA: 0s - loss: 0.7388 - accuracy: 0.6726

122/122 [==============================] - 65s 330ms/step - loss: 0.7388 - accuracy: 0.6726 - val_loss: 0.6504 - val_accuracy: 0.7237
Epoch 2/50
121/122 [============================>.] - ETA: 0s - loss: 0.5336 - accuracy: 0.7748

122/122 [==============================] - 39s 317ms/step - loss: 0.5332 - accuracy: 0.7750 - val_loss: 0.5793 - val_accuracy: 0.7464
Epoch 3/50
121/122 [============================>.] - ETA: 0s - loss: 0.3999 - accuracy: 0.8435

122/122 [==============================] - 37s 307ms/step - loss: 0.3996 - accuracy: 0.8437 - val_loss: 0.5536 - val_accuracy: 0.7608
Epoch 4/50
121/122 [============================>.] - ETA: 0s - loss: 0.3315 - accuracy: 0.8802

122/122 [==============================] - 37s 308ms/step - loss: 0.3318 - accuracy: 0.8798 - val_loss: 0.5417 - val_accuracy: 0.7567
Epoch 5/50
122/122 [==============================] - 22s 181ms/step - loss: 0.2708 - accuracy: 0.9167 - val_loss: 0.5631 - val_accuracy: 0.7588
Epoch 6/50
122/122 [==============================] - 23s 187ms/step - loss: 0.2179 - accuracy: 0.9422 - val_loss: 0.5489 - val_accuracy: 0.7588
Epoch 7/50
121/122 [============================>.] - ETA: 0s - loss: 0.1782 - accuracy: 0.9595

122/122 [==============================] - 39s 319ms/step - loss: 0.1780 - accuracy: 0.9595 - val_loss: 0.5276 - val_accuracy: 0.7608
Epoch 8/50
122/122 [==============================] - 22s 181ms/step - loss: 0.1392 - accuracy: 0.9778 - val_loss: 0.5395 - val_accuracy: 0.7649
Epoch 9/50
122/122 [==============================] - 23s 185ms/step - loss: 0.1103 - accuracy: 0.9897 - val_loss: 0.5425 - val_accuracy: 0.7608
Epoch 10/50
122/122 [==============================] - 22s 182ms/step - loss: 0.0876 - accuracy: 0.9933 - val_loss: 0.5937 - val_accuracy: 0.7526
Epoch 11/50
122/122 [==============================] - 22s 179ms/step - loss: 0.0718 - accuracy: 0.9956 - val_loss: 0.5415 - val_accuracy: 0.7649
Epoch 12/50
122/122 [==============================] - 22s 184ms/step - loss: 0.0568 - accuracy: 0.9974 - val_loss: 0.5739 - val_accuracy: 0.7649


In [ ]:
train_acc_hist = hist.history['accuracy']
train_acc = 0
count = 0
for i in train_acc_hist:
  train_acc += i
  count += 1
train_acc = train_acc / count
print("Training accuracy was: ", str(train_acc))

val_acc_hist = hist.history['val_accuracy']
val_acc = 0
count = 0
for i in val_acc_hist:
  val_acc += i
  count += 1
val_acc = val_acc / count
print("Validation accuracy was: ", str(val_acc))

Training accuracy was:  0.9119367102781931
Validation accuracy was:  0.7561855614185333


In [ ]:
#Roll back to model found performing best on validation set during training.
double_conv_classifier.load_weights("./BERT_conv_model_doubleblock")

#Generate predictions using this model
predictions = double_conv_classifier.predict(X_test)

y_pred = []
for pred in predictions:
  label = np.argmax(pred)
  y_pred.append(label)

y_real = []
for encoding in y_test:
  label = np.argmax(encoding)
  y_real.append(label)

#Evaluate performance of the model using the current set of hyperparameters
evaluation = mce.evaluate_performance(y_pred, y_real)

print("Tested model with 2 convolutional layers on the test set")
for metric in evaluation:
  print(metric, ": ", evaluation[metric], "\n")
print("\n \n \n")

16/16 [==============================] - 3s 153ms/step
Tested model with 2 convolutional layers on the test set
Accuracy :  80.0 

Base Positive :  {'TP': 90, 'FP': 34, 'TN': 321, 'FN': 40} 

Base Neutral :  {'TP': 255, 'FP': 51, 'TN': 149, 'FN': 30} 

Base Negative :  {'TP': 43, 'FP': 12, 'TN': 403, 'FN': 27} 

Advanced Positive :  {'Precision': 0.7258064516129032, 'Recall': 0.6923076923076923, 'Specificity': 0.9042253521126761} 

Advanced Neutral :  {'Precision': 0.8333333333333334, 'Recall': 0.8947368421052632, 'Specificity': 0.745} 

Advanced Negative :  {'Precision': 0.7818181818181819, 'Recall': 0.6142857142857143, 'Specificity': 0.9710843373493976} 

Balanced Accuracy :  0.7337767495662232 

F_Score :  0.4330611035115569 


 
 



### Adding an additional convolutional layer does not significantly increase the performance. To not unessecarily increase the model's complexity, we choose to keep one single convolutional layer.

# Test the final hyperparameter set on the test set to record results

In [ ]:
final_model = create_BERT_CNN(bert_preprocess, bert_encoder, kernel_size=1, filters=256, activation="relu", padding="same")

optim_adam = tf.keras.optimizers.Adam(learning_rate=0.0003)
final_model.compile(loss='categorical_crossentropy', optimizer=optim_adam, metrics=['accuracy'])

early_stop =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./final_BERT_conv_model", save_best_only=True)

print("Training the final BERT+CNN model... \n \n")

hist = final_model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop, checkpoint_cb])

train_acc_hist = hist.history['accuracy']
train_acc = 0
count = 0
for i in train_acc_hist:
  train_acc += i
  count += 1
train_acc = train_acc / count
print("Training accuracy was: ", str(train_acc))

val_acc_hist = hist.history['val_accuracy']
val_acc = 0
count = 0
for i in val_acc_hist:
  val_acc += i
  count += 1
val_acc = val_acc / count
print("Validation accuracy was: ", str(val_acc))

Training the final BERT+CNN model... 
 

Epoch 1/50
122/122 [==============================] - ETA: 0s - loss: 0.7531 - accuracy: 0.6664

122/122 [==============================] - 47s 343ms/step - loss: 0.7531 - accuracy: 0.6664 - val_loss: 0.6659 - val_accuracy: 0.6948
Epoch 2/50
121/122 [============================>.] - ETA: 0s - loss: 0.5531 - accuracy: 0.7627

122/122 [==============================] - 55s 450ms/step - loss: 0.5527 - accuracy: 0.7629 - val_loss: 0.6005 - val_accuracy: 0.7464
Epoch 3/50
121/122 [============================>.] - ETA: 0s - loss: 0.4261 - accuracy: 0.8352

122/122 [==============================] - 37s 304ms/step - loss: 0.4259 - accuracy: 0.8354 - val_loss: 0.5714 - val_accuracy: 0.7485
Epoch 4/50
121/122 [============================>.] - ETA: 0s - loss: 0.3598 - accuracy: 0.8740

122/122 [==============================] - 39s 321ms/step - loss: 0.3598 - accuracy: 0.8738 - val_loss: 0.5465 - val_accuracy: 0.7546
Epoch 5/50
122/122 [==============================] - 28s 229ms/step - loss: 0.3014 - accuracy: 0.9040 - val_loss: 0.5639 - val_accuracy: 0.7402
Epoch 6/50
121/122 [============================>.] - ETA: 0s - loss: 0.2556 - accuracy: 0.9282

122/122 [==============================] - 60s 491ms/step - loss: 0.2554 - accuracy: 0.9283 - val_loss: 0.5216 - val_accuracy: 0.7588
Epoch 7/50
122/122 [==============================] - 25s 202ms/step - loss: 0.2192 - accuracy: 0.9461 - val_loss: 0.5294 - val_accuracy: 0.7546
Epoch 8/50
122/122 [==============================] - 22s 182ms/step - loss: 0.1811 - accuracy: 0.9621 - val_loss: 0.5271 - val_accuracy: 0.7588
Epoch 9/50
122/122 [==============================] - 24s 197ms/step - loss: 0.1549 - accuracy: 0.9737 - val_loss: 0.5386 - val_accuracy: 0.7485
Epoch 10/50
122/122 [==============================] - 23s 187ms/step - loss: 0.1320 - accuracy: 0.9843 - val_loss: 0.5307 - val_accuracy: 0.7670
Epoch 11/50
122/122 [==============================] - 22s 176ms/step - loss: 0.1111 - accuracy: 0.9892 - val_loss: 0.5260 - val_accuracy: 0.7567
Training accuracy was:  0.8932826681570574
Validation accuracy was:  0.7480787309733304


In [ ]:
#Roll back to model found performing best on validation set during training.
final_model.load_weights("./final_BERT_conv_model")

#Generate predictions using this model
predictions = final_model.predict(X_test)

y_pred = []
for pred in predictions:
  label = np.argmax(pred)
  y_pred.append(label)

y_real = []
for encoding in y_test:
  label = np.argmax(encoding)
  y_real.append(label)

#Evaluate performance of the model using the current set of hyperparameters
evaluation = mce.evaluate_performance(y_pred, y_real)

print("Tested final model on the test set")
for metric in evaluation:
  print(metric, ": ", evaluation[metric], "\n")
print("\n \n \n")

16/16 [==============================] - 3s 162ms/step
Tested final model on the test set
Accuracy :  77.9381 

Base Positive :  {'TP': 96, 'FP': 49, 'TN': 306, 'FN': 34} 

Base Neutral :  {'TP': 239, 'FP': 44, 'TN': 156, 'FN': 46} 

Base Negative :  {'TP': 43, 'FP': 14, 'TN': 401, 'FN': 27} 

Advanced Positive :  {'Precision': 0.6620689655172414, 'Recall': 0.7384615384615385, 'Specificity': 0.8619718309859155} 

Advanced Neutral :  {'Precision': 0.8445229681978799, 'Recall': 0.8385964912280702, 'Specificity': 0.78} 

Advanced Negative :  {'Precision': 0.7543859649122807, 'Recall': 0.6142857142857143, 'Specificity': 0.9662650602409638} 

Balanced Accuracy :  0.7304479146584409 

F_Score :  0.40840716375139613 


 
 



In [ ]:
print(mce.confusion_matrix(y_pred, y_real))

[[ 43   7   7]
 [ 17 239  27]
 [ 10  39  96]]


In [ ]:
#Save the trained model
final_model.save("/content/drive/My Drive/Bachelor Scriptie KI/Programming/Notebooks/financial_BERT_CNN_model")

#Sentence-level error inspection

In [ ]:
#Load the trained model
load_model = tf.keras.models.load_model("/content/drive/My Drive/Bachelor Scriptie KI/Programming/Notebooks/financial_BERT_CNN_model")

In [ ]:
load_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text_input[0][0]']             
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [ ]:
#Generate predictions using this model
predictions_loaded = load_model.predict(X_test)

y_pred_loaded = []
for pred in predictions_loaded:
  label = np.argmax(pred)
  y_pred_loaded.append(label)

y_real_loaded = []
for encoding in y_test:
  label = np.argmax(encoding)
  y_real_loaded.append(label)

#Evaluate performance of the model using the current set of hyperparameters
evaluation_loaded = mce.evaluate_performance(y_pred_loaded, y_real_loaded)

print("Tested final model on the test set")
for metric in evaluation_loaded:
  print(metric, ": ", evaluation_loaded[metric], "\n")
print("\n \n \n")

16/16 [==============================] - 11s 183ms/step
Tested final model on the test set
Accuracy :  77.9381 

Base Positive :  {'TP': 96, 'FP': 49, 'TN': 306, 'FN': 34} 

Base Neutral :  {'TP': 239, 'FP': 44, 'TN': 156, 'FN': 46} 

Base Negative :  {'TP': 43, 'FP': 14, 'TN': 401, 'FN': 27} 

Advanced Positive :  {'Precision': 0.6620689655172414, 'Recall': 0.7384615384615385, 'Specificity': 0.8619718309859155} 

Advanced Neutral :  {'Precision': 0.8445229681978799, 'Recall': 0.8385964912280702, 'Specificity': 0.78} 

Advanced Negative :  {'Precision': 0.7543859649122807, 'Recall': 0.6142857142857143, 'Specificity': 0.9662650602409638} 

Balanced Accuracy :  0.7304479146584409 

F_Score :  0.40840716375139613 


 
 



In [ ]:
print(mce.confusion_matrix(y_pred_loaded, y_real_loaded))

[[ 43   7   7]
 [ 17 239  27]
 [ 10  39  96]]


In [ ]:
def get_specific_errors(dataframe, y_pred, y_real, vertical, horizontal):
	"""
	Get the indexes from specific cells in the confusion matrix.

	param dataframe: Pandas DataFrame containing the sentences and indices.
	param y_pred: (list) contains the predicted sentiments.
	param y_real: (list) contains the real sentiments.
	param vertical: (int) corresponds to the column in the confusion matrix.
	param horizontal: (int) corresponds to the row in the confusion matrix.
	"""
	i = 0
	errors = []
	while i < len(dataframe.Sentence):
		if (horizontal == y_pred[i]) and (vertical == y_real[i]):
			errors.append(dataframe.index[i])
		i += 1
	return errors

In [ ]:
#Neutral sentences predicted as positive.
pos_neu_errors_index = get_specific_errors(test, y_pred_loaded, y_real_loaded, 1, 2)

In [ ]:
with open("/content/drive/MyDrive/Bachelor Scriptie KI/Programming/Notebooks/Error Indexes/Financial/updated_DL_financial_pos_neu.txt", "w") as writefile:
  for index in pos_neu_errors_index:
    writefile.write(str(index))
    writefile.write("\n")

In [ ]:
#Positive sentences predicted as neutral.
neu_pos_errors_index = get_specific_errors(test, y_pred_loaded, y_real_loaded, 2, 1)

In [ ]:
with open("/content/drive/MyDrive/Bachelor Scriptie KI/Programming/Notebooks/Error Indexes/Financial/updated_DL_financial_neu_pos.txt", "w") as writefile:
  for index in neu_pos_errors_index:
    writefile.write(str(index))
    writefile.write("\n")

In [ ]:
#True positives for the neutral class.
tp_neu_index = get_specific_errors(test, y_pred_loaded, y_real_loaded, 1, 1)

In [ ]:
with open("/content/drive/MyDrive/Bachelor Scriptie KI/Programming/Notebooks/Error Indexes/Financial/updated_DL_financial_tp_neu.txt", "w") as writefile:
  for index in tp_neu_index:
    writefile.write(str(index))
    writefile.write("\n")

In [ ]:
#Negative sentences predicted as neutral.
neu_neg_errors_index = get_specific_errors(test, y_pred_loaded, y_real_loaded, 0, 1)

In [ ]:
with open("/content/drive/MyDrive/Bachelor Scriptie KI/Programming/Notebooks/Error Indexes/Financial/updated_DL_financial_neu_neg.txt", "w") as writefile:
  for index in neu_neg_errors_index:
    writefile.write(str(index))
    writefile.write("\n")

In [ ]:
#Positive sentences predicted as negative.
pos_neg_errors_index = get_specific_errors(test, y_pred_loaded, y_real_loaded, 0, 2)

In [ ]:
with open("/content/drive/MyDrive/Bachelor Scriptie KI/Programming/Notebooks/Error Indexes/Financial/updated_DL_financial_pos_neg.txt", "w") as writefile:
  for index in pos_neg_errors_index:
    writefile.write(str(index))
    writefile.write("\n")